See [Corum Contractor Report](https://apps.dtic.mil/dtic/tr/fulltext/u2/a282289.pdf)

Velocity factor is given by:

$$V_f = \frac{1}{\sqrt{1 + 20\left(\frac{D}{s}\right)^{2.5} \left(\frac{D}{\lambda_o}\right)^{0.5}}}$$

* $\lambda_o$ free space wavelength
* $D$ helix diameter
* $s$ turn to turn spacing

All in the same units

Schelkunoff's transmission line characteristic impedance:

$$Z_o = \frac{60}{V_f} \left[ ln\left( \frac{4 H}{D}\right) - 1 \right]$$

* $H$ is the physical height of the helix


In [9]:
import math
import numpy as np
import matplotlib.pyplot as plt

inch = 2.54/100
ft = 12*inch
cm = 1/100
MHz = 1e6
speed_of_light = 3e8

## Red Resonator

In [15]:
D = 2.38
l = 3.75 - (1/16)
r = D/2
N = 107*l
l_w = math.pi*D*N*inch

L_total = ((r*N)**2)*1e-6/(9*r + 10*l)
fo = 1.529e6 # Measured frequency
l_w, L_total, fo

(74.93355778534603, 0.004632927056591165, 1529000.0)

In [16]:
D = 2.38 * inch
s = (1/107.0) * inch
lo = speed_of_light/fo
H = (3.75 - (1/16))*inch

V_f = 1/math.sqrt(1 + 20*((D/s)**2.5)*((D/lo)**0.5))
V_f

0.0016590491790251997

In [17]:
Z_o = (60/V_f)*(math.log(4*H/D) - 1)
Z_o

29805.358594132213

In [18]:
L_dis = L_total/l_w
C_dis = L_dis/(Z_o**2)
(L_dis, C_dis)

(6.182713317660167e-05, 6.959698193348055e-14)

Loss:

$$\alpha l = \frac{7.8125 \left(\frac{H}{D} \right)^{0.2}}{d_w Z_o \sqrt{f_{MHz}}}$$

* $d_w$ is wire diameter in inches

Base impedance:

$$R_{base} \approx \alpha l Z_o $$

$$R_{base} \approx \frac{7.8125 \left(\frac{H}{D} \right)^{0.2}}{d_w \sqrt{f_{MHz}}} $$


In [19]:
d_w = s / inch
a1 = 7.8125 * ((H/D)**0.2)
a2 = d_w * Z_o * math.sqrt(fo/1e6)
alpha_l = a1/a2

R_base = alpha_l * Z_o
f'Base impedance: {R_base}'

'Base impedance: 737.9060646650111'

In [25]:
1/alpha_l

40.39180597826231

### Electrical Length

Electrical length of the line in radians is

$$\theta = \frac{2 \pi H}{V_f \lambda_o}$$

In [24]:
theta = 2*math.pi*H/(V_f * lo)
theta

103.58460567213221

### Step Up

$$\left\lvert \frac{V_{top}}{V_{base}} \right\rvert = \frac{\sqrt{[1 + \lvert \Gamma_2 \rvert cos \phi]^2 + [\lvert \Gamma_2 \rvert sin \phi]^2}}{\sqrt{[e^{\alpha l} cos \theta + \lvert \Gamma_2 \rvert e^{-\alpha l} cos(\phi-\theta)]^2 + [e^{\alpha l} sin \theta + \lvert \Gamma_2 \rvert e^{-\alpha l} sin(\phi-\theta)]^2}}$$

In [31]:
# This needs work
gamma_2 = 1
theta = phi = math.pi/2
a = math.sqrt((1+gamma_2*math.cos(phi))**2 + (gamma_2*math.sin(phi))**2)
t1 = (math.cos(theta)*(math.e**alpha_l) + math.cos(phi-theta)*gamma_2*(math.e**(-alpha_l)))**2
t2 = (math.sin(theta)*(math.e**alpha_l) + math.sin(phi-theta)*gamma_2*(math.e**(-alpha_l)))**2
b = math.sqrt(t1 + t2)
step_up = a/b
step_up

0.9993875043080072

In [35]:
math.e**alpha_l

1.0250665082884522

In [38]:
math.exp(alpha_l)

1.0250665082884522

## Measurements

The following measurements were made on 10/4/2020.

* Base drive 10V p-p or 5V amplitude
* Generator impedance of $50 \Omega$
* Resonator base voltage is 4.45 V

LTSpice simulation results compare well with measurements using the following LTRA model:

* ```.model myTline LTRA(R=10 L=6.183e-05 C=6.6e-14 len=80.9)```
* Add Spice ltline component and set Value to myTline

This simulation suggests the following:

* $R_{base} = 409 \Omega$
* $V_{base} = 4.45 V$
* $V_{top} = 338 V$
* ${SWR} = 76$

A comparable ideal transmission line model can be achieved with these parameters:

* ```Td=0.164u Z0=29805```
* $R_{top} = 2.25 M\Omega$

### Correction Factor

Measured base resistance is lower than theoretical estimate.

In [7]:
C_f = 409/R_base
C_f

0.5542710916540233

In [8]:
fo = 717e3
D = 2.38 * inch
s = (1/107.0) * inch
lo = speed_of_light/fo
H = (3.75 - (1/16))*inch

V_f = 1/math.sqrt(1 + 20*((D/s)**2.5)*((D/lo)**0.5))

Z_o = (60/V_f)*(math.log(4*H/D) - 1)

d_w = s / inch
a1 = 7.8125 * ((H/D)**0.2)
a2 = d_w * Z_o * math.sqrt(fo/1e6)
alpha_l = a1/a2

R_base = C_f*alpha_l * Z_o
f'Base impedance: {R_base}'

'Base impedance: 597.2655600592714'

## RF Helical Resonator

[Wikipedia article](https://en.wikipedia.org/wiki/Helical_resonator), original [RF Cafe article](http://www.rfcafe.com/references/electrical/helical-resonator.htm)

$Q=35.9\cdot d\cdot {\sqrt  {f}}$

$Z_{o}=\frac{136190}{d\cdot f}$

$h=1.5\cdot d$

* $Q$ - quality factor (dimensionless)
* $Z_{o}$ - resonator characteristic impedance (Ohms)
* $d$ - mean helix diameter (cm)
* $h$ - height of helix (cm)
* $f$ - frequency (MHz)

In [13]:
d = D/cm
h = H/cm
f = fo/MHz

Q = 35.9 * d * math.sqrt(f)
Zo = 136190/(d*f)
Q, Zo

(183.76580445189515, 31420.66631284676)

## Power Estimate

In [16]:
R_top = 2.25e6
V_top = 50e3
P50 = (V_top**2)/R_top
on_time = 25e-6
cycle_time = 1/300
P50*on_time/cycle_time

8.333333333333334

In [17]:
w_o = 2*math.pi*fo
Q = R_top/(w_o*L_total)
Q

50.552158246960644

In [18]:
1/(0.1e-6 * w_o**2)

1.0834909607019656e-07

## Tesla Resonator

In [26]:

fo = 94e3
D = 8.25 * ft
s = 1 * inch
lo = speed_of_light/fo
H = 106*inch

V_f = 1/math.sqrt(1 + 20*((D/s)**2.5)*((D/lo)**0.5))
V_f

0.004273839070920192

In [27]:
Z_o = (60/V_f)*(math.log(4*H/D) - 1)
Z_o

6382.275011578561

In [23]:
math.log(2.718)

0.999896315728952